In [ ]:
%%capture
%pip install transformers trl

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
out = pipe(' ',num_return_sequences = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_texts = []

for i in out:
  generated_texts.append(i['generated_text'])

In [ ]:
generated_texts


[' \n\n"The fact that I\'ve been asked to write about this issue has been a huge burden to me. It has been a huge burden to me because I know that there are people out there who are really upset by this, that I\'m not going to be able to deal with that as a writer. It is a sad day for me because I\'ve tried to write about this, so I want to say thank you for that. I\'m sorry, I\'m not sure how to respond to that."\n\nAsked about the decision to publish the article on Twitter, Tod added, "I think it\'s important to note that this isn\'t a discussion about the merits or moral of the topic, this is a conversation about a person\'s perspective on this issue, and I\'m not happy where I\'m coming from."\n\nWhat do you think about the decision to publish in a blog post that had already been shared more than 100,000 times?\n\n[Image via screengrab]\n\n——\n\n>> Follow Evan McMurry (@evanmcmurry) on Twitter\n\nHave a tip we should know? tips@mediaite.com',
 '  and the rest of the top-most row.\n

In [ ]:
from datasets import load_dataset
ds = load_dataset("alapoodle/nyp-headlines")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

headlines.txt: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2240 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2240
    })
})

In [ ]:
ds['train']['text'][8]

'Schools plan to replace Styrofoam lunch trays'

In [ ]:
pipe(ds['train']['text'][8])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Schools plan to replace Styrofoam lunch trays with fiber-based plastic bags to help them save money.\n\nThe plans include replacing the plastic bags with a fiber-based plastic bag, and replacing the plastic bags with a plastic bag that is coated with an adhesive that is softer than the plastic bag\'s plastic.\n\n"The plastic bags are like a sponge that keeps the plastic from getting into the food system," said Scott McCaffrey, a senior vice president with Food & Water Watch, based in San Diego.\n\nThe plastic bags are less likely to break down, reduce food waste and prevent food-borne illnesses.\n\nIn January, a city study found that Styrofoam bags are more likely to be contaminated than regular plastic bags.\n\n"That\'s why it\'s important to use it," said Mary Gaudet, a research associate at the National School of Public Health School of Public Health. "It\'s easier to clean and it\'s more cost-effective."\n\nThe plastic bags are more likely to have a higher recy

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer, SFTConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("alapoodle/nyp-headlines")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
training_params = SFTConfig(
    output_dir="./results",
    num_train_epochs = 3, # how many times we iterate over the dataset as a whole
    learning_rate = 2e-4, # how many "steps" we take in adjusting the parameters to make up for loss
    weight_decay = 0.001, # way of regularizing the parameters
    dataset_text_field = "text",
    report_to="none", # this is a new param, to avoid a login to W&B
    bf16 = False,
    fp16 = False
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    processing_class = tokenizer,
    args = training_params
)

Adding EOS to train dataset:   0%|          | 0/2240 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2240 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2240 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,4.877400
20,4.715000
30,4.647600
40,4.627900
50,4.641400
60,4.669900
70,4.969200
80,4.640600
90,4.563200
100,4.624500


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=840, training_loss=2.9262299957729523, metrics={'train_runtime': 2877.5606, 'train_samples_per_second': 2.335, 'train_steps_per_second': 0.292, 'total_flos': 71089144086528.0, 'train_loss': 2.9262299957729523, 'epoch': 3.0})

In [ ]:
# commenting out this code so that it doesn't run (when I create this website)!

# ## first, save the model to a folder called "models"
trainer.model.save_pretrained("models")
trainer.tokenizer.save_pretrained("models")

# ## then, load our model from that folder
model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")

# ## create a pipe() function that calls our new model
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=100)

# ### run inference!
pipe("Cats")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cpu


[{'generated_text': "Cats and dogs eat plastic trash in NYC's forests"}]